In [2]:
# Import library NLTK untuk preprocessing
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [6]:
# Import modul dari library tensorflow untuk melatih model ANN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
# from sklearn.preprocessing import LabelBinarizer

# Import library pendukung lainnya
import numpy as np
import random
import json
import pickle

In [7]:
# Reading the JSON file
# we will use json.loads() for this and
# store the content of the file into a variable named 'intents' (type: dict)
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [8]:
# Pre-Processing
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
      # tokenize each and every word in the sentence
      w = nltk.word_tokenize(pattern)
      # add word to the words list
      words.extend(w)
      # add word(s) to documents
      documents.append((w, intent['tag']))
      # add tags to our classes list
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Perform steming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

112 documents
30 classes ['aktivitas', 'apa_kabar', 'apresiasi', 'balasan_salam', 'berita', 'covid19', 'cuaca', 'ekspresi_kagum', 'google', 'hina', 'identitas', 'inspirasi', 'karan', 'kontak', 'kriket', 'lagu', 'lelucon', 'obrolan_menyenangkan', 'opsi', 'pembuat', 'perpisahan', 'sapaan', 'saran', 'tanggal_waktu', 'teka_teki', 'terima_kasih', 'tertawa', 'tidak', 'usia', 'waktu']
132 unique stemmed words ['10', '19', 'ab', 'ad', 'aku', 'ap', 'at', 'bag', 'bagaiman', 'baik', 'baik-baik', 'bantu', 'ber', 'berap', 'bergun', 'berit', 'bias', 'bis', 'bodoh', 'buat', 'buruk', 'ceritak', 'covid', 'cuac', 'dadah', 'denganmu', 'di', 'dia', 'diam', 'dibu', 'ditawark', 'dukung', 'giman', 'googl', 'hah', 'hai', 'halo', 'har', 'heb', 'hei', 'hol', 'hubung', 'idiot', 'in', 'ind', 'inspirasimu', 'internet', 'itu', 'jam', 'jump', 'kab', 'kabarmu', 'kabarny', 'kamu', 'kap', 'kar', 'kasih', 'ker', 'kriket', 'lag', 'lagu', 'lain', 'lakuk', 'lelucon', 'luar', 'lucu', 'makasih', 'malik', 'membantu', 'membuat

In [12]:
# Create training data
training = []
# Create an empty array for output
output_empty = [0] * len(classes)

# Create training set, bag of words for each sentences
for doc in documents:
    # Initialize bag of words
    bag = []
    # List of tokenized words for the pattern and
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    # Create bag of words array
    for w in words:
        bag.append(1 if w in pattern_words else 0)

    # Output is '1' for current tag and '0' for rest of other tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffling features and turning it into array using numpy
random.shuffle(training)
training = np.array(training, dtype=object)

# Creating training lists
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [14]:
print(train_x[:5],'\n')
print(train_y[:5])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [16]:
# Save preprocessing
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", "wb"))

In [18]:
# Build model
model = Sequential()
model.add(Dense(32, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Compile model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_x, train_y, epochs=200, batch_size=8, validation_split=0.2, verbose=1)

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.0482 - loss: 3.4093 - val_accuracy: 0.0000e+00 - val_loss: 3.3797
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0840 - loss: 3.3811 - val_accuracy: 0.0000e+00 - val_loss: 3.3707
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1307 - loss: 3.3560 - val_accuracy: 0.0000e+00 - val_loss: 3.3625
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1497 - loss: 3.3371 - val_accuracy: 0.0000e+00 - val_loss: 3.3546
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1383 - loss: 3.3096 - val_accuracy: 0.0000e+00 - val_loss: 3.3461
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2369 - loss: 3.2861 - val_accuracy: 0.0435 - val_loss: 3.3353
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2151 - loss: 3.2543 - val_accuracy: 0.0435 - val_loss: 3.3243
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2978 - loss: 3.2201 - v

In [22]:
# Save model
model.save("chatbot_model.h5")

In [24]:
# load the saved model
from tensorflow.keras.models import load_model
model = load_model("chatbot_model.h5")

# Restoring all the data structures
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']

In [26]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                # if show_details:
                    # print("found in bag: %s"% w)

    return(np.array(bag))

In [28]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    input_data = np.expand_dims(bow(sentence, words), axis=0)
    results = model.predict([bow(sentence, words).reshape(1, -1)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # Loop as long as there are matches to process
        for i in intents['intents']:
            # find a tag matching the first result
            if i['tag'] == results[0][0]:
                # a random response from the intent
                return random.choice(i['responses'])

    return "Maaf, saya tidak mengerti."

In [30]:
# Interact with simple chatbot
print("Tekan 0 jika Anda tidak ingin mengobrol dengan chatbot kami.")
while True:
    message = input("You: ")
    if message == "0":
        break
    result = response(message)
    print("Bot:", result)

Tekan 0 jika Anda tidak ingin mengobrol dengan chatbot kami.


You:  Siapa kamu?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 132))',)
  warnings.warn(msg)


Bot: Saya Ted, chatbot berbasis Deep Learning.


...

You:  Semoga harimu menyenangkan


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Bot: Saya juga senang ngobrol denganmu! Jangan lupa kembali lagi ya!


You:  Tidak ada


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Bot: Hai, ada yang bisa saya bantu?


You:  0
